In [7]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v -O3')

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# GOD I AM SO HAPPY, SO HAPPY, THAT JUPYTER IN REMOTE SSH IS WORKING AGAIN! LETS DO SOME SMEM STUFF BABYYYYYY. 

In [32]:
%%cuda

#include<stdio.h> 
#include<stdlib.h>
#include<cuda.h>
#include<cuda_runtime.h> 

constexpr int N_iter = 100000;
constexpr int N_warps = 32;
constexpr int N_lanes = 32;
constexpr int N_blocks = 1;

__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

__global__ void smem_no_bc()
{
  __shared__ float A[N_warps][N_lanes];
  int warp_id = threadIdx.y; 
  int lane_id = threadIdx.x; 
  
  float* ptr = &A[warp_id][lane_id];
  int addr = (int)ptr & 0xFFFF; //#0x is just a prefix meaning its hexadecimal 
  //# each F is a 15, and each digit is worth that power of 16. 
  //# in fact this number is 2^16 + 1, which means we are bit masking with the and 
  //# and extracting the first 16 digits of our pointer. but why? 
  /*
  """
  well really, each address is worth a byte (8 bits) and so how many bytes do we have in this shared memory?
  we have 8*32 bytes, so we have 8*8*32 bits, so its enough for our adress to be that big 
  well 8*8*32 = 2^6 * 2^5 = 2^11, 
  
  but the adress itself, if you do addr + 1, you are moving 8 bits (or 1 byte) 
  so then, if we have 2^16 - 1 addrs, we are guuci, but idk whats more to this nvm
  """
  */
  float r1;
  int n_clocks = 0;
  unsigned long long start;
  unsigned long long end;
  
  
  for (int j = 0; j < N_iter; j++) {
      
      start = get_clock64();
      
      asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr));
      
      end = get_clock64();
      
      if (threadIdx.x == 0 && threadIdx.y == 0)
        {
          n_clocks += end - start;
        }
  }
  
  __syncthreads(); 
  if (threadIdx.x == 0 && threadIdx.y == 0)
  {
    printf("%d", n_clocks);
  }
  
}

int main()
{
   
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

 
  cudaEventRecord(start, 0);

 
  //#weirdly enough, it is actually threadidx.x that has stride one, even tho its 
  //#on the left of the tuple. so we gotta be careful about that launch
  dim3 threads(N_lanes, N_warps);
  dim3 grid(N_blocks);
  
  smem_no_bc<<<grid, threads>>>();


  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("GPU Kernel time: %f ms\n", milliseconds);
  
  
  /*
  """
  so how do we calculate our memory transfer speed, well, we have N_warps warps, 
  so per iteration, we issue N_warp loads per bank, so our speed will be 
  N_loads/bank/second. 
  so for N_iter iterations, 
  smem_bank_speed = (N_warps*N_iter)/time 
  """
  */
  float seconds = milliseconds*0.001;
  float mega = 1000000.0;
  printf("mega_loads/bank/second = %f \n", (N_warps*N_iter)/(seconds*mega));
  
  int deviceId;
  cudaGetDevice(&deviceId);
  
  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, deviceId);
  

  double smClockMHz = prop.clockRate / 1000.0;
  
  printf("Device: %s\n", prop.name);
  printf("Base SM Clock Rate: %.2f MHz\n", smClockMHz);
}

2711953GPU Kernel time: 31.770304 ms
mega_loads/bank/second = 100.722984 
Device: NVIDIA GeForce RTX 5090
Base SM Clock Rate: 2407.00 MHz



In [18]:
"""
one wonders why I am not getting 1 load per bank per clock cycle. 

"""

'\none wonders why I am not getting 1 load per bank per clock cycle. \n\n'

In [28]:
%%cuda

#include <stdio.h>
#include <cuda_runtime.h>

// A device function to read the 64-bit clock register.
// The "l" specifier in the asm constraint tells the compiler to use a 64-bit register.
__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

// A kernel to measure the cycles taken by a specific operation
__global__ void measure_work(unsigned long long* start_cycles, unsigned long long* stop_cycles, int iterations) {
    // Only have one thread record the time to avoid writing from multiple threads
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        
        *start_cycles = get_clock64();

        // --- The work you want to measure goes here ---
        // For example, a loop with some math operations
        float x = 1.0f;
        for (int i = 0; i < iterations; ++i) {
            x = sinf(x);
        }

        *stop_cycles = get_clock64();
        // Use the result to prevent the compiler from optimizing the loop away
        if (x < -2.0f) *start_cycles = 0;
    }
}

int main() {
    // Pinned host memory for faster transfers, and device memory
    unsigned long long *h_start, *h_stop;
    unsigned long long *d_start, *d_stop;
    cudaHostAlloc(&h_start, sizeof(unsigned long long), cudaHostAllocDefault);
    cudaHostAlloc(&h_stop, sizeof(unsigned long long), cudaHostAllocDefault);
    cudaMalloc(&d_start, sizeof(unsigned long long));
    cudaMalloc(&d_stop, sizeof(unsigned long long));

    // CUDA events for measuring wall-clock time
    cudaEvent_t start_event, stop_event;
    cudaEventCreate(&start_event);
    cudaEventCreate(&stop_event);

    int work_iterations = 1000000;

    // Record start time
    cudaEventRecord(start_event, 0);

    // Launch kernel
    measure_work<<<1, 1>>>(d_start, d_stop, work_iterations);

    // Record stop time and synchronize
    cudaEventRecord(stop_event, 0);
    cudaEventSynchronize(stop_event);

    // Copy results from device to host
    cudaMemcpy(h_start, d_start, sizeof(unsigned long long), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_stop, d_stop, sizeof(unsigned long long), cudaMemcpyDeviceToHost);

    // Calculate elapsed time and cycles
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start_event, stop_event);
    double seconds = milliseconds / 1000.0;
    unsigned long long cycles_elapsed = *h_stop - *h_start;

    // Calculate the actual, measured SM frequency during the kernel run
    double measured_frequency_ghz = (cycles_elapsed / seconds) / 1e9;
    
    printf("Wall time: %.6f s\n", seconds);
    printf("SM cycles elapsed: %llu\n", cycles_elapsed);
    printf("Measured SM Frequency during kernel execution: %.3f GHz\n", measured_frequency_ghz);

    // Cleanup
    cudaFree(d_start);
    cudaFree(d_stop);
    cudaFreeHost(h_start);
    cudaFreeHost(h_stop);
    cudaEventDestroy(start_event);
    cudaEventDestroy(stop_event);

    return 0;
}


Wall time: 0.033058 s
SM cycles elapsed: 96000558
Measured SM Frequency during kernel execution: 2.904 GHz

